Notebook for chEMBL Database curation and generation of a diverset set of small molecule for structure in BOLTZ2


In [ ]:
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import Fingerprints
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import sys
import sqlite3
import pandas as pd
from tqdm.auto import tqdm
from molbloom import buy
import os
from typing import Optional, Tuple
import numpy as np


In [ ]:
# Define the database location and UNIPROT mappings
db_location = 'chembl_36/chembl_36_sqlite/chembl_36.db'
uniprot_mapping = 'chembl_uniprot_mapping.txt'

# Default location of the ChEMBL database file
DEFAULT_DB_FILENAME = "chembl_36/chembl_36_sqlite/chembl_36.db"

# Load the UNIPROT mappings
uniprot_df = pd.read_csv(uniprot_mapping, sep='\t')
print("UNIPROT mappings loaded:")




Edit Cell below for different filtering options

In [ ]:
########################################################
# Query for drug-like molecules (Lipinski-like)
# Edit the values below to change the query parameters
mw_range = (500, 900)  # MW between 200-500
logp_range = (3, 5)    # LogP between 0-5
purchasable_only = True  # Only include purchasable compounds
create_random = True
random_subset_amount = 200




In [ ]:


# Functions for querying the database

def get_property_query() -> str:
    """Returns the SQL query to fetch compounds based on molecular weight and LogP."""
    sql = """SELECT
        cs.molregno,
        cs.canonical_smiles,
        cp.mw_freebase as molecular_weight,
        cp.alogp,
        cp.hba,
        cp.hbd,
        cp.psa,
        cp.rtb,
        cp.num_ro5_violations,
        md.chembl_id
    FROM
        compound_structures cs
            JOIN
        compound_properties cp ON cs.molregno = cp.molregno
            JOIN
        molecule_dictionary md ON cs.molregno = md.molregno
    WHERE
        cp.mw_freebase BETWEEN ? AND ?
        AND cp.alogp BETWEEN ? AND ?
        AND cs.canonical_smiles IS NOT NULL;"""
    return sql

def query_by_properties(
    mw_range: Tuple[float, float],
    logp_range: Tuple[float, float],
    db_filename: str = DEFAULT_DB_FILENAME
) -> pd.DataFrame:
    """
    Query ChEMBL database for compounds within specified MW and LogP ranges.
    
    Parameters:
    -----------
    mw_range : tuple of (min_mw, max_mw)
        Molecular weight range
    logp_range : tuple of (min_logp, max_logp)
        LogP range
    db_filename : str
        Path to ChEMBL database file
        
    Returns:
    --------
    pd.DataFrame
        DataFrame containing compounds matching the criteria
    """
    if not os.path.exists(db_filename):
        print(f"Error: Can't find ChEMBL database file {db_filename}", file=sys.stderr)
        return pd.DataFrame()

    min_mw, max_mw = mw_range
    min_logp, max_logp = logp_range

    print(f"Querying compounds with MW: {min_mw}-{max_mw}, LogP: {min_logp}-{max_logp}")

    try:
        conn = sqlite3.connect(db_filename)
        sql = get_property_query()
        df = pd.read_sql_query(sql, conn, params=(min_mw, max_mw, min_logp, max_logp))
        conn.close()
    except sqlite3.Error as e:
        print(f"Database error: {e}", file=sys.stderr)
        return pd.DataFrame()

    print(f"Found {len(df)} compounds matching criteria")
    return df

def add_purchasability(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add purchasability information to the DataFrame using MolBloom.
    """
    if df.empty:
        return df
    
    print("Checking for purchasable compounds...")
    df['purchasable'] = [buy(smi, canonicalize=True) for smi in tqdm(df.canonical_smiles)]
    print(f"{df['purchasable'].sum()} of {len(df)} compounds are purchasable")
    return df



# Check to see if there is a is a pkl file already, if so, load it, if not run the query_by_properties function
pkl_file = f"compounds_MW{mw_range[0]}-{mw_range[1]}_LogP{logp_range[0]}-{logp_range[1]}.pkl"

if os.path.exists(pkl_file):
    df_compounds = pd.read_pickle(pkl_file)
    print(f"Loaded DataFrame from {pkl_file}")
else:
    df_compounds = query_by_properties(mw_range, logp_range, db_filename=db_location)

# Write df_compounds to a .pkl file

if not df_compounds.empty:
    pkl_file = f"compounds_MW{mw_range[0]}-{mw_range[1]}_LogP{logp_range[0]}-{logp_range[1]}.pkl"
    df_compounds.to_pickle(pkl_file)
    print(f"DataFrame saved to {pkl_file}")


# Display summary statistics
if not df_compounds.empty:
    print("\n=== Summary Statistics ===")
    print(df_compounds[['molecular_weight', 'alogp', 'hba', 'hbd', 'psa']].describe())
    
    print("\n=== Sample compounds ===")
    print(df_compounds.head(10))
    
    # Optionally add purchasability check (this may take a while for large datasets)
    # use conditional check to see if purchasability column already exists
    if 'purchasable' not in df_compounds.columns:
        df_compounds = add_purchasability(df_compounds)
    
    if purchasable_only:
        df_compounds = df_compounds[df_compounds['purchasable']]
        print(f"\nFiltered to {len(df_compounds)} purchasable compounds.")
    # Save to CSV
    output_file = f"compounds_MW{mw_range[0]}-{mw_range[1]}_LogP{logp_range[0]}-{logp_range[1]}.csv"
    df_compounds.to_csv(output_file, index=False)
    df_compounds.to_pickle(pkl_file)

    print(f"\nResults saved to {output_file}")
    print(f"DataFrame saved to {pkl_file}")

In [ ]:
# Take the smiles strings and convert them to RDKit molecules and generate fingerprints
if not df_compounds.empty:
    if 'rdkit_mol' not in df_compounds.columns or 'fingerprint' not in df_compounds.columns:
        print("\nGenerating RDKit molecules and fingerprints...")
    
        df_compounds['rdkit_mol'] = df_compounds['canonical_smiles'].apply(AllChem.MolFromSmiles)
        df_compounds['fingerprint'] = df_compounds['rdkit_mol'].apply(lambda mol: AllChem.GetMorganFingerprintAsBitVect(mol, 2) if mol is not None else None)

    # Alternatively, use RDKFingerprint
    # df_compounds['fingerprint'] = df_compounds['rdkit_mol'].apply(lambda mol: Chem.RDKFingerprint(mol) if mol is not None else None)  
        print("RDKit molecules and fingerprints generated.")

# Update the .pkl file with the new columns
if not df_compounds.empty:
    pkl_file = f"compounds_MW{mw_range[0]}-{mw_range[1]}_LogP{logp_range[0]}-{logp_range[1]}.pkl"
    df_compounds.to_pickle(pkl_file)
    print(f"Updated DataFrame with RDKit molecules and fingerprints saved to {pkl_file}")




In [ ]:
# Use the morgan fingerprints for tSNE visualization and k-means clustering

if not df_compounds.empty:
    print("\nPerforming tSNE and k-means clustering...")
    # Extract fingerprints and convert to numpy array
    fps = [fp for fp in df_compounds['fingerprint'] if fp is not None]
    fps_array = np.array([np.array(fp) for fp in fps])

    # Perform tSNE
    tsne = TSNE(n_components=2, random_state=42)
    tsne_results = tsne.fit_transform(fps_array)

    # Perform k-means clustering
    n_clusters = 20
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(fps_array)

    # Add tSNE results and cluster labels to DataFrame
    df_compounds = df_compounds[df_compounds['fingerprint'].notnull()].copy()
    df_compounds['tsne_x'] = tsne_results[:, 0]
    df_compounds['tsne_y'] = tsne_results[:, 1]
    df_compounds['cluster'] = cluster_labels

    # Calculate cluster statistics to identify tight clusters
    from scipy.spatial.distance import cdist
    
    cluster_stats = []
    for cluster_id in range(n_clusters):
        cluster_mask = df_compounds['cluster'] == cluster_id
        cluster_points = fps_array[cluster_mask]
        
        if len(cluster_points) > 1:
            # Calculate cluster center (centroid)
            cluster_center = kmeans.cluster_centers_[cluster_id]
            
            # Calculate distances from each point to cluster center
            distances = cdist([cluster_center], cluster_points, metric='euclidean')[0]
            
            # Calculate cluster tightness metrics
            mean_distance = np.mean(distances)
            std_distance = np.std(distances)
            max_distance = np.max(distances)
            
            cluster_stats.append({
                'cluster_id': cluster_id,
                'size': len(cluster_points),
                'mean_distance': mean_distance,
                'std_distance': std_distance,
                'max_distance': max_distance,
                'tightness_score': mean_distance + std_distance  # Lower is tighter
            })
    
    cluster_stats_df = pd.DataFrame(cluster_stats)
    
    # Define threshold for "tight" clusters (you can adjust this)
    tightness_threshold = cluster_stats_df['tightness_score'].quantile(0.3)  # Top 30% tightest clusters
    tight_clusters = cluster_stats_df[cluster_stats_df['tightness_score'] <= tightness_threshold]
    
    print(f"\nIdentified {len(tight_clusters)} tight clusters out of {n_clusters} total clusters")
    print("\nTight cluster statistics:")
    print(tight_clusters[['cluster_id', 'size', 'mean_distance', 'std_distance']])
    
    # Select one representative molecule from each tight cluster
    representative_molecules = []
    tight_cluster_ids = set()
    
    for _, cluster_info in tight_clusters.iterrows():
        cluster_id = int(cluster_info['cluster_id'])  # Convert to int
        tight_cluster_ids.add(cluster_id)
        
        cluster_mask = df_compounds['cluster'] == cluster_id
        cluster_compounds = df_compounds[cluster_mask].copy()
        cluster_fps = fps_array[cluster_mask]
        
        # Calculate distance of each molecule to cluster center
        cluster_center = kmeans.cluster_centers_[cluster_id]
        distances = cdist([cluster_center], cluster_fps, metric='euclidean')[0]
        
        # Add distances to dataframe
        cluster_compounds['distance_to_center'] = distances
        
        # Select the molecule closest to center
        representative = cluster_compounds.nsmallest(1, 'distance_to_center')
        representative_molecules.append(representative)
    
    # Combine all representative molecules
    df_representatives = pd.concat(representative_molecules, ignore_index=True)
    
    print(f"\nSelected {len(df_representatives)} representative molecules from tight clusters")
    
    # Get all molecules NOT in tight clusters
    df_not_in_tight_clusters = df_compounds[~df_compounds['cluster'].isin(tight_cluster_ids)].copy()
    
    print(f"Kept {len(df_not_in_tight_clusters)} molecules not in tight clusters")
    
    # Combine representatives with non-tight-cluster molecules
    df_diverse = pd.concat([df_representatives, df_not_in_tight_clusters], ignore_index=True)
    
    print(f"\n=== Diversity Selection Summary ===")
    print(f"Original compounds: {len(df_compounds)}")
    print(f"Molecules in tight clusters (removed): {len(df_compounds[df_compounds['cluster'].isin(tight_cluster_ids)]) - len(df_representatives)}")
    print(f"Representatives from tight clusters: {len(df_representatives)}")
    print(f"Molecules not in tight clusters: {len(df_not_in_tight_clusters)}")
    print(f"Final diverse set: {len(df_diverse)}")
    
    # Plot tSNE results showing what was kept vs removed
    plt.figure(figsize=(14, 6))
    
    # Left plot: Original with tight clusters highlighted
    plt.subplot(1, 2, 1)
    plt.scatter(df_compounds['tsne_x'], df_compounds['tsne_y'], 
                c='lightgrey', alpha=0.3, s=30, label='All compounds')
    
    # Highlight tight clusters
    for cluster_id in tight_cluster_ids:
        cluster_mask = df_compounds['cluster'] == cluster_id
        plt.scatter(df_compounds[cluster_mask]['tsne_x'], 
                   df_compounds[cluster_mask]['tsne_y'],
                   alpha=0.6, s=50, label=f'Tight cluster {cluster_id}')
    
    # Mark representative molecules
    plt.scatter(df_representatives['tsne_x'], df_representatives['tsne_y'],
                c='red', marker='*', s=300, edgecolors='black', linewidths=2,
                label='Representatives', zorder=5)
    
    plt.title(f'Original: {len(df_compounds)} compounds')
    plt.xlabel('tSNE X')
    plt.ylabel('tSNE Y')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    
    # Right plot: Final diverse set
    plt.subplot(1, 2, 2)
    plt.scatter(df_diverse['tsne_x'], df_diverse['tsne_y'], 
                c='blue', alpha=0.5, s=50, label='Diverse set')
    plt.scatter(df_representatives['tsne_x'], df_representatives['tsne_y'],
                c='red', marker='*', s=300, edgecolors='black', linewidths=2,
                label='Cluster representatives', zorder=5)
    
    plt.title(f'Diverse Set: {len(df_diverse)} compounds')
    plt.xlabel('tSNE X')
    plt.ylabel('tSNE Y')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Replace df_compounds with the diverse set
    df_compounds = df_diverse.copy()
    
    # Save the diverse set
    output_file = f"diverse_MW{mw_range[0]}-{mw_range[1]}_LogP{logp_range[0]}-{logp_range[1]}.csv"
    df_compounds.to_csv(output_file, index=False)
    
    pkl_file = f"diverse_MW{mw_range[0]}-{mw_range[1]}_LogP{logp_range[0]}-{logp_range[1]}.pkl"
    df_compounds.to_pickle(pkl_file)
    
    print(f"\nDiverse compound set saved to {output_file}")
    print(f"Diverse compound set saved to {pkl_file}")
    
    # Also save just the representatives for reference
    repr_output_file = f"representatives_MW{mw_range[0]}-{mw_range[1]}_LogP{logp_range[0]}-{logp_range[1]}.csv"
    df_representatives.to_csv(repr_output_file, index=False)
    print(f"Cluster representatives saved to {repr_output_file}")
    
    # Display sample from diverse set
    print("\n=== Sample Diverse Compounds ===")
    print(df_compounds[['chembl_id', 'canonical_smiles', 'cluster']].head(10))

# Save the final diverse set as a CSV file:
    output_file = f"diverse_MW{mw_range[0]}-{mw_range[1]}_LogP{logp_range[0]}-{logp_range[1]}.csv"
    df_compounds.to_csv(output_file, index=False)
    
    pkl_file = f"diverse_MW{mw_range[0]}-{mw_range[1]}_LogP{logp_range[0]}-{logp_range[1]}.pkl"
    df_compounds.to_pickle(pkl_file)
    
    print(f"\nDiverse compound set saved to {output_file}")
    print(f"Diverse compound set saved to {pkl_file}")
    
    # Also save just the representatives for reference
    repr_output_file = f"representatives_MW{mw_range[0]}-{mw_range[1]}_LogP{logp_range[0]}-{logp_range[1]}.csv"
    df_representatives.to_csv(repr_output_file, index=False)
    print(f"Cluster representatives saved to {repr_output_file}")
    
    


In [ ]:
# Optional: Create a random subset of the diverse set

if not df_compounds.empty:
    if len(df_compounds) > random_subset_amount:
        print(f"\nCreating random subset of {random_subset_amount} molecules from {len(df_compounds)} diverse compounds...")
        df_random_subset = df_compounds.sample(n=random_subset_amount, random_state=42)
        
        # Save the random subset
        output_file = f"random{random_subset_amount}_MW{mw_range[0]}-{mw_range[1]}_LogP{logp_range[0]}-{logp_range[1]}.csv"
        df_random_subset.to_csv(output_file, index=False)
        
        pkl_file = f"random{random_subset_amount}_MW{mw_range[0]}-{mw_range[1]}_LogP{logp_range[0]}-{logp_range[1]}.pkl"
        df_random_subset.to_pickle(pkl_file)
        
        print(f"Random subset saved to {output_file}")
        print(f"Random subset saved to {pkl_file}")
        
        # Display sample
        print("\n=== Sample Random Subset ===")
        print(df_random_subset[['chembl_id', 'canonical_smiles', 'molecular_weight', 'alogp']].head(10))
    else:
        print(f"\nDataFrame has {len(df_compounds)} molecules, which is less than or equal to requested subset size of {random_subset_amount}")
        print("Skipping random subset creation.")
